In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import epiweeks

In [2]:
# Initialize Browser
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST driver version for 96.0.4664
Driver [C:\Users\Drew\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [3]:
# Visit gender data source url
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTLO0Kf-G8sP1If_Fhj5GapXHSchZ3OwLJmGq6fF2h9-gE8rHrNxyhD_nm4Oa4YicmhtZ3MLcBoQxbH/pubhtml?wmode=opaque#"
browser.visit(url)

time.sleep(1)

In [4]:
# Initialize column list
columns = ['location','date','cases_total','male_cases',
           'female_cases','percent_male_cases',
           'percent_female_cases','male_case_rate',
           'female_case_rate','deaths_total','male_deaths',
           'female_deaths','percent_male_deaths',
           'percent_female_deaths','male_death_rate',
           'female_death_rate']

In [5]:
# Scrape site and return a dataframe with the displayed week's data
def scrape_weekly_data():
    
    # Scrap into soup
    html = browser.html
    soup = bs(html, "html.parser")
    
    displayed_table = soup.find('div', style='position: relative;')
    
    # Initialize week's dataframe
    week_df = pd.DataFrame(columns=columns)
    
    # Traverse rows 5-57 containing data cells
    for row_index in range(5,58):
        row = displayed_table.find('div', text=row_index).parent.parent
        
        # Get data from relevant cells
        cells = row.find_all('td')[0:16]
    
        # Add row to end of dataframe
        week_df.loc[len(week_df)] = cells
    
    return week_df

In [6]:
scrape_weekly_data()

,location,date,cases_total,male_cases,female_cases,percent_male_cases,percent_female_cases,male_case_rate,female_case_rate,deaths_total,male_deaths,female_deaths,percent_male_deaths,percent_female_deaths,male_death_rate,female_death_rate
0,[Alabama],[4/13],"[4,572]",[1876],[2615],[41.03],[57.20],[79.63],[104.24],[96],[56],[40],[58.80],[41.20],[2.36],[1.58]
1,[Alaska],[4/13],[272],[134],[138],[49.30],[50.70],[34.78],[39.07],[8],[],[],[],[],[0.00],[0.00]
2,[Arizona],[4/13],[3539],[1663],[1876],[47.00],[53.00],[48.16],[53.69],[169],[100],[69],[59.00],[41.00],[2.89],[1.98]
3,[Arkansas],[4/13],[1280],[552],[728],[43.10],[56.90],[37.57],[47.84],[27],[],[],[],[],[0.00],[0.00]
4,[California ],[4/13],"[21,794]",[10926],[10571],[50.13],[48.50],[56.16],[53.67],[651],[],[],[],[],[0.00],[0.00]
5,[Colorado ],[4/13],[7303],[3451],[3814],[47.25],[52.23],[124.09],[138.69],[290],[178],[111],[61.38],[38.38],[6.40],[4.05]
6,[Connecticut ],[4/13],[16809],[8852],[7654],[52.66],[45.54],[506.66],[417.25],[1036],[545],[485],[52.61],[46.81],[31.19],[26.44]
7,[Delaware],[4/13],[1625],[],[],[],[],[0.00],[0.00],[35],[],[],[],[],[0.00],[0.00]
8,[[District of Columbia]],[4/13],[1875],[967],[902],[51.57],[48.11],[297.65],[250.82],[50],[],[],[],[],[0.00],[0.00]
9,[Florida],[4/13],[19895],[],[],[],[],[0.00],[0.00],[461],[],[],[],[],[0.00],[0.00]


In [7]:
week_buttons = browser.find_by_tag('ul').first.find_by_tag('li')